In [151]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [152]:
cd C:/Users/0105l/Desktop/데이터/

C:\Users\0105l\Desktop\데이터


In [154]:
def distance(x1, y1, x2, y2):
    m = (y2-y1) / (x2-x1)
    return m

In [155]:
# 포지션 => 기울기로 변환
# coef.shape = (300,1)
def Coefficient(df): 
    coef = []
    for i in range(49, 351):
        frame1 = df[i][gradlab1][0:2]
        frame2 = df[i][gradlab2][0:2]
        coef.append(distance(frame1[0], frame1[1], frame2[0], frame2[1]))
    coef = pd.DataFrame(coef, columns= ['coef'])
    
    q3=coef['coef'].quantile(0.75)
    q1=coef['coef'].quantile(0.25)
    iqr=q3-q1
    outlier_top=q3+1.5*iqr
    outlier_bottom=q1-1.5*iqr
    idx=coef[(coef['coef']<outlier_bottom) | (coef['coef']>outlier_top)].index
    
    idx_list=[]
    for i in idx:
        idx_list.append(str(i))
    idx_list.append('end')

    idx_string = ' '.join(idx_list)
    idx_string_split = idx_string.split('end')
    outlier_str = idx_string_split[0].strip()
    outlier = list(map(int, outlier_str.split(' ')))

    for i in range(len(coef)-1):
        if i in outlier:
            if i == 0:
                continue
            elif i != (len(coef)-1):
                coef['coef'][i] = (coef['coef'][i-1] + coef['coef'][i+1])/2
            else:
                break
                
    coef = np.array(coef)[1:301]
    
    return coef

In [156]:
# 가속도 넘파이 이상치 변환
# acc_array.shape = (300,2)
def Acceleration(pos_1, array_1, acclab):
    df = pd.DataFrame(array_1[:,acclab,:], columns=['x','y','z'])
    idx_list=[]
    for i in ['x','y','z']:
        q3=df[i].quantile(0.75)
        q1=df[i].quantile(0.25)
        iqr=q3-q1
        outlier_top=q3+1.5*iqr
        outlier_bottom=q1-1.5*iqr

        idx=df[(df[i]<outlier_bottom) | (df[i]>outlier_top)].index
        for i in idx:
            idx_list.append(str(i))
        idx_list.append('end')
    
    idx_string = ' '.join(idx_list)
    idx_string_split = idx_string.split('end')
    x_outlier_str = idx_string_split[0].strip()
    y_outlier_str = idx_string_split[1].strip()
    x_outlier = list(map(int, x_outlier_str.split(' ')[:-1]))
    y_outlier = list(map(int, y_outlier_str.split(' ')[:-1]))
    
    acc_1_parsed = array_1[:,acclab,:]
    for i in range(len(pos_1)-1):
        if i in x_outlier:
            acc_1_parsed[i][0] = (acc_1_parsed[i-1][0] + acc_1_parsed[i+1][0])/2 
        if i in y_outlier:
            acc_1_parsed[i][1] = (acc_1_parsed[i-1][1] + acc_1_parsed[i+1][1])/2 
       
    final_df = pd.DataFrame(acc_1_parsed, columns = ['x','y','z'])
    acc_array = StandardScaler().fit_transform(final_df)
    acc_array = acc_array[49:349,:]
    
    return acc_array[:,:2] 

In [157]:
#부위 별로 모델을 만들거라 상관없다 ..? (오답 데이터 가져오는 거 생각할 필요 있음) 
hand = 'Right'
gesture_list = ['URF','UMF','ULF','DLF','DMF','DRF','URB','UMB','ULB','DLB','DMB','DRB','URO','ULO','DLO','DRO']
right_gesture = gesture_list[0]
num_session = 5
#총 세션 개수 = batch size
#5로 적어야 1~4까지 들어감

acclab1 = 8
acclab2 = 12
gradlab1 = 8
gradlab2 = 12


In [158]:
def dataloader(gesture):
    data2 = np.zeros((1,300,6))
    for i in range(1, num_session):
        data = np.zeros((300,1))
        pos = np.load('%s_%s_p_%d.npy' % (hand, gesture, i))
        acc = np.load('%s_%s_a_%d.npy' % (hand, gesture, i))
        acc1 = Acceleration(pos, acc, acclab1)
        acc2 = Acceleration(pos, acc, acclab2)
        coef = Coefficient(pos)
        if gesture == right_gesture:
            label = np.ones((300,1))
        else:
            label = np.zeros((300,1))
        data = np.hstack((data,acc1,acc2,coef,label))
        data = data[:, 1:].reshape((1,300,6))
        data2 = np.vstack((data2,data))
    
        
    return data2

In [159]:
target_data = dataloader('URF')
error_data = dataloader('DLB')

In [169]:
type(target_data)

numpy.ndarray

In [160]:
target_data.shape, error_data.shape

((5, 300, 6), (5, 300, 6))

In [175]:
error_data[0]

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [176]:
data = np.vstack((target_data[1:], error_data[1:]))
data

array([[[-3.29066519e-01, -2.47648078e-01, -1.07156350e-01,
         -1.71393815e-01,  1.24951225e+00,  1.00000000e+00],
        [-1.27500655e+00, -7.55905290e-01, -5.62856579e-01,
         -5.59727408e-01,  1.32934755e+00,  1.00000000e+00],
        [ 1.28950053e+00,  6.89500970e-01, -2.22177874e-03,
         -1.01876799e-03,  1.27165818e+00,  1.00000000e+00],
        ...,
        [ 1.46416609e-01,  3.70244551e-01,  5.29748617e-01,
          6.03417302e-01, -1.84926575e+01,  1.00000000e+00],
        [-1.88858184e-01, -1.65248434e-01,  6.23319775e-01,
          3.90512374e-01, -7.95013185e+00,  1.00000000e+00],
        [ 8.20031876e-01,  9.93098851e-01,  3.03370629e-01,
          1.30073630e-01, -3.22716929e+00,  1.00000000e+00]],

       [[ 4.62174576e-01,  6.19516186e-01,  2.13561525e-01,
          3.92794339e-01,  1.77478833e+00,  1.00000000e+00],
        [-1.70099406e-01, -4.32923348e-02,  1.17690016e+00,
          6.55550482e-01,  3.07311352e+00,  1.00000000e+00],
        [ 3.77370

In [177]:
data.shape

(8, 300, 6)

In [180]:
import torch
from torch.utils.data import random_split
train_data, test_data = random_split(data, [6, 2])

In [182]:
len(train_data)
len(test_data)

2

In [183]:
train_data = np.vstack((data[0], data[1], data[2], data[4], data[5], data[6]))
test_data = np.vstack((data[3], data[7]))

In [189]:
train_data = train_data.reshape(6, 300, 6)

In [190]:
test_data = test_data.reshape(2, 300, 6)

In [191]:
x_train, y_train = train_data[:, :, :5], train_data[:, :, -1]
x_test, y_test = test_data[:, :, :5], test_data[:, :, -1]

In [283]:
y_train = [1, 1, 1, 1, 0, 0]
y_test = [1, 0]

In [284]:
x_train = torch.Tensor(x_train)
y_train = torch.Tensor(y_train)
x_test = torch.Tensor(x_test)
y_test = torch.Tensor(y_test)

In [285]:
x_train.shape

torch.Size([6, 300, 5])

In [286]:
x_test.shape

torch.Size([2, 300, 5])

In [319]:
y_train = y_train.reshape(6, 1)

In [288]:
y_train

tensor([1., 1., 1., 1., 0., 0.])

In [289]:
y_test.shape

torch.Size([2])

In [320]:
input = torch.randn(100,3,3) #(batch, seq_len_, input_size)
input_size= 5
#seq_length = 300
hidden_size = 256
layers = 2
dropout_rate= 0.5
num_classes = 1

In [335]:
class LSTM_Chicka(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, dropout_rate):
        super(LSTM_Chicka,self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.dropout_rate = dropout_rate
        self.lstm =  nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout_rate, batch_first = True)
        #self.layer_1 = nn.Linear(hidden_size, 64)
        #self.layer_2 = nn.Linear(256,64)
        #self.layer_3 = nn.Linear(64,16)
        self.layer_out = nn.Linear(hidden_size, num_classes) 
        #self.relu = nn.ReLU()  
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        #h_0 = torch.tensor(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) # x.size(0) batch size 
        #c_0 = torch.tensor(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        #output, (hn,cn) = self.lstm(x,(h_0,c_0))
        out, _status = self.lstm(x)
        
        
        #hn = hn.view(-1, self.hidden_size)
        #out = self.relu(hn)
        #out = self.layer_1(out) 
        #out = self.relu(out) 
        #out = self.layer_2(out)
        #out = self.relu(out)
        #out = self.layer_3(out)
        #out = self.relu(out)
        out = self.layer_out(out[:, -1]) 
        out = self.sigmoid(out)

        return out

In [363]:
model = LSTM_Chicka(num_classes, input_size, hidden_size, num_layers, dropout_rate)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer= optimizer)

In [353]:
outputs = model(x_train)
print(outputs.shape)

torch.Size([6, 1])


In [354]:
outputs

tensor([[0.4975],
        [0.4981],
        [0.4997],
        [0.4964],
        [0.4982],
        [0.4958]], grad_fn=<SigmoidBackward0>)

In [368]:
x_train.shape

torch.Size([6, 300, 5])

In [364]:
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(x_train)
    loss = criterion(outputs.view(-1, outputs.shape[-1]), y_train)
    loss.backward()
    optimizer.step()
    #if epoch % 10 == 0:
    print("Epoch: %d, loss: %1.5f" % (epoch+1, loss.item()))

Epoch: 1, loss: 0.68569
Epoch: 2, loss: 0.67414
Epoch: 3, loss: 0.66544
Epoch: 4, loss: 0.64684
Epoch: 5, loss: 0.63414
Epoch: 6, loss: 0.60296
Epoch: 7, loss: 0.56130
Epoch: 8, loss: 0.51137
Epoch: 9, loss: 0.49981
Epoch: 10, loss: 0.43651
Epoch: 11, loss: 0.39520
Epoch: 12, loss: 0.36498
Epoch: 13, loss: 0.33374
Epoch: 14, loss: 0.30876
Epoch: 15, loss: 1.21624
Epoch: 16, loss: 0.29241
Epoch: 17, loss: 0.29257
Epoch: 18, loss: 0.28866
Epoch: 19, loss: 0.28580
Epoch: 20, loss: 0.27939
Epoch: 21, loss: 0.27610
Epoch: 22, loss: 0.27234
Epoch: 23, loss: 0.26280
Epoch: 24, loss: 0.26575
Epoch: 25, loss: 0.25089
Epoch: 26, loss: 0.24046
Epoch: 27, loss: 0.23078
Epoch: 28, loss: 0.21943
Epoch: 29, loss: 0.20197
Epoch: 30, loss: 0.18751
Epoch: 31, loss: 0.15526
Epoch: 32, loss: 0.14490
Epoch: 33, loss: 0.12637
Epoch: 34, loss: 0.08121
Epoch: 35, loss: 0.03414
Epoch: 36, loss: 0.25848
Epoch: 37, loss: 0.02471
Epoch: 38, loss: 0.02668
Epoch: 39, loss: 0.02720
Epoch: 40, loss: 0.85505
Epoch: 41

In [365]:
predict = model(x_test)
print(predict.shape)

torch.Size([2, 1])


In [366]:
predict

tensor([[0.3360],
        [0.0076]], grad_fn=<SigmoidBackward0>)